# Imports,  Settings and Paths

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd#
%matplotlib inline
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
from Code.Models.c7o2h10_model import DeepPotential, train, normalize, backtransform

In [3]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
model_path = './ModelCheckpoints/c7o2h10/'

In [4]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [5]:
use_cuda = torch.cuda.is_available()
print('use cuda:  ', use_cuda)

use cuda:   True


# Read the Data

In [6]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1
Y_npy, Y_min, Y_max = normalize(Y_npy)
X_npy.shape

(404000, 19, 72)

### Check for NaN values

In [7]:
use_ids = np.where(np.isnan(X_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [8]:
X_npy = X_npy[use_ids]
Y_npy = Y_npy[use_ids]
X_npy.shape

(404000, 19, 72)

In [9]:
X_npy = X_npy
Y_npy = Y_npy

### Split into test and train

In [10]:
X_train_npy, X_test_npy = X_npy[train_ids], X_npy[test_ids]
Y_train_npy, Y_test_npy = Y_npy[train_ids], Y_npy[test_ids]
print('X Train shape: {}\tX Test shape: {}\nY Train shape: {}\tY Test shape: {}'.format(X_train_npy.shape,
                                                                                        X_test_npy.shape,
                                                                                        Y_train_npy.shape,
                                                                                        Y_test_npy.shape))

X Train shape: (400000, 19, 72)	X Test shape: (4000, 19, 72)
Y Train shape: (400000,)	Y Test shape: (4000,)


### Shuffle the Dataset

In [11]:
shuffle = np.arange(X_train_npy.shape[0])
np.random.shuffle(shuffle)
X_train_npy = X_train_npy[shuffle]
Y_train_npy = Y_train_npy[shuffle]

In [12]:
X_train_npy[0]

array([[ 2.10233263e-01,  3.49171333e-01,  9.97571883e-01, ...,
         6.12323400e-17,  7.85524974e-01,  6.18829957e-01],
       [ 2.93159804e-01,  4.86902015e-01,  3.18215446e-01, ...,
         6.12323400e-17,  1.00000000e+00, -3.82212013e-17],
       [ 2.40715223e-01,  6.62701965e-01,  9.57767296e-01, ...,
         5.61260652e-01, -1.84688302e-02,  9.99829437e-01],
       ...,
       [ 1.92179425e-01, -4.63619570e-01,  9.81894146e-01, ...,
         6.12323400e-17,  7.68310079e-01,  6.40077826e-01],
       [ 1.95178508e-01,  3.91669819e-01,  8.24680127e-01, ...,
         6.12323400e-17,  7.97912431e-01,  6.02773384e-01],
       [ 2.33265249e-01, -5.62286930e-01,  7.89416049e-01, ...,
         3.31413248e-01,  9.01617992e-01,  4.32533232e-01]])

In [13]:
X_train_npy[shuffle][0]

array([[ 1.85009448e-01, -2.08366548e-01,  9.81626252e-01, ...,
         2.99388022e-02,  4.53053455e-01,  8.91483352e-01],
       [ 2.55418609e-01, -1.78658433e-01,  1.46413822e-01, ...,
        -9.73142243e-01, -9.93068812e-01, -1.17534401e-01],
       [ 2.50558060e-01, -4.78492845e-01,  9.95265229e-01, ...,
         5.78364764e-01,  7.56811133e-01,  6.53633620e-01],
       ...,
       [ 2.30767898e-01, -3.91629229e-01,  9.75996164e-01, ...,
        -7.20451711e-01,  9.00514629e-01, -4.34825716e-01],
       [ 1.96973693e-01, -1.59680979e-01,  9.33796448e-01, ...,
         3.36103348e-01,  5.98502756e-01,  8.01120747e-01],
       [ 1.93931417e-01,  2.91288135e-01,  7.40085814e-01, ...,
         6.12323400e-17,  1.00000000e+00,  0.00000000e+00]])

### Create Pytorch Variables

In [14]:
if use_cuda:
    X_train = Variable(torch.Tensor(X_train_npy).cuda())
    Y_train = Variable(torch.Tensor(Y_train_npy).cuda(), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy).cuda())
    Y_test = Variable(torch.Tensor(Y_test_npy).cuda(), requires_grad=False)
else:
    X_train = Variable(torch.Tensor(X_train_npy))
    Y_train = Variable(torch.Tensor(Y_train_npy), requires_grad=False)
    X_test = Variable(torch.Tensor(X_test_npy))
    Y_test = Variable(torch.Tensor(Y_test_npy), requires_grad=False)

/home/mi/stefaah94/miniconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU1 NVS 310 which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


# Train Model

## Do the learning

In [ ]:
if use_cuda:
    init_deep_pot = DeepPotential().cuda()
else:
    init_deep_pot = DeepPotential()
init_optim = torch.optim.Adam(init_deep_pot.parameters())
deep_pot, optim = train(init_deep_pot, init_optim, X_train, Y_train,
                        X_test, Y_test,
                        300, 128, (0.01, 0.96, 1.5),
                        checkpoint_path=model_path, print_every=100)

total: 0.0 %	current epoch: 3.2 %	loss: 0.035514	time estimate: 668.5 min
total: 0.0 %	current epoch: 6.4 %	loss: 0.010216	time estimate: 765.8 min
total: 0.0 %	current epoch: 9.6 %	loss: 0.007503	time estimate: 747.0 min
total: 0.0 %	current epoch: 12.8 %	loss: 0.006186	time estimate: 714.2 min
total: 0.1 %	current epoch: 16.0 %	loss: 0.005237	time estimate: 710.9 min
total: 0.1 %	current epoch: 19.2 %	loss: 0.004629	time estimate: 740.5 min
total: 0.1 %	current epoch: 22.4 %	loss: 0.004179	time estimate: 759.2 min
total: 0.1 %	current epoch: 25.6 %	loss: 0.004309	time estimate: 749.8 min
total: 0.1 %	current epoch: 28.8 %	loss: 0.003748	time estimate: 734.6 min
total: 0.1 %	current epoch: 32.0 %	loss: 0.003721	time estimate: 730.1 min
total: 0.1 %	current epoch: 35.2 %	loss: 0.003329	time estimate: 722.2 min
total: 0.1 %	current epoch: 38.4 %	loss: 0.003193	time estimate: 712.9 min
total: 0.1 %	current epoch: 41.6 %	loss: 0.002985	time estimate: 712.1 min
total: 0.1 %	current epoch: 

total: 1.2 %	current epoch: 51.2 %	loss: 0.001259	time estimate: 734.5 min
total: 1.2 %	current epoch: 54.4 %	loss: 0.001271	time estimate: 733.3 min
total: 1.2 %	current epoch: 57.6 %	loss: 0.001298	time estimate: 732.0 min
total: 1.2 %	current epoch: 60.8 %	loss: 0.001254	time estimate: 731.5 min
total: 1.2 %	current epoch: 64.0 %	loss: 0.00122	time estimate: 732.0 min
total: 1.2 %	current epoch: 67.2 %	loss: 0.001252	time estimate: 732.7 min
total: 1.2 %	current epoch: 70.4 %	loss: 0.00118	time estimate: 733.6 min
total: 1.2 %	current epoch: 73.6 %	loss: 0.001269	time estimate: 734.2 min
total: 1.3 %	current epoch: 76.8 %	loss: 0.00122	time estimate: 734.6 min
total: 1.3 %	current epoch: 80.0 %	loss: 0.001249	time estimate: 735.5 min
total: 1.3 %	current epoch: 83.2 %	loss: 0.001164	time estimate: 735.8 min
total: 1.3 %	current epoch: 86.4 %	loss: 0.001229	time estimate: 734.6 min
total: 1.3 %	current epoch: 89.6 %	loss: 0.001185	time estimate: 733.4 min
total: 1.3 %	current epoch: 

total: 2.3 %	current epoch: 99.2 %	loss: 0.000831	time estimate: 731.0 min
saved checkpoint at: ./ModelCheckpoints/c7o2h10/epoch_6

total: 2.3 %	current epoch: 3.2 %	loss: 0.000829	time estimate: 731.5 min
total: 2.4 %	current epoch: 6.4 %	loss: 0.00082	time estimate: 730.8 min
total: 2.4 %	current epoch: 9.6 %	loss: 0.000813	time estimate: 730.2 min
total: 2.4 %	current epoch: 12.8 %	loss: 0.000817	time estimate: 730.3 min
total: 2.4 %	current epoch: 16.0 %	loss: 0.000848	time estimate: 730.5 min
total: 2.4 %	current epoch: 19.2 %	loss: 0.000808	time estimate: 730.8 min
total: 2.4 %	current epoch: 22.4 %	loss: 0.000807	time estimate: 730.7 min
total: 2.4 %	current epoch: 25.6 %	loss: 0.000859	time estimate: 730.8 min
total: 2.4 %	current epoch: 28.8 %	loss: 0.000866	time estimate: 731.2 min
total: 2.4 %	current epoch: 32.0 %	loss: 0.000819	time estimate: 731.5 min
total: 2.5 %	current epoch: 35.2 %	loss: 0.000805	time estimate: 732.0 min
total: 2.5 %	current epoch: 38.4 %	loss: 0.0008

total: 3.5 %	current epoch: 44.8 %	loss: 0.000588	time estimate: 744.5 min
total: 3.5 %	current epoch: 48.0 %	loss: 0.000573	time estimate: 744.2 min
total: 3.5 %	current epoch: 51.2 %	loss: 0.000588	time estimate: 744.3 min
total: 3.5 %	current epoch: 54.4 %	loss: 0.000636	time estimate: 744.3 min
total: 3.5 %	current epoch: 57.6 %	loss: 0.000583	time estimate: 744.6 min
total: 3.5 %	current epoch: 60.8 %	loss: 0.000603	time estimate: 744.9 min
total: 3.5 %	current epoch: 64.0 %	loss: 0.000592	time estimate: 744.5 min
total: 3.6 %	current epoch: 67.2 %	loss: 0.000567	time estimate: 744.7 min
total: 3.6 %	current epoch: 70.4 %	loss: 0.00057	time estimate: 744.4 min
total: 3.6 %	current epoch: 73.6 %	loss: 0.000576	time estimate: 743.9 min
total: 3.6 %	current epoch: 76.8 %	loss: 0.000567	time estimate: 743.5 min
total: 3.6 %	current epoch: 80.0 %	loss: 0.000569	time estimate: 743.7 min
total: 3.6 %	current epoch: 83.2 %	loss: 0.000573	time estimate: 743.7 min
total: 3.6 %	current epoch

total: 4.6 %	current epoch: 92.8 %	loss: 0.000447	time estimate: 728.4 min
total: 4.7 %	current epoch: 96.0 %	loss: 0.000439	time estimate: 728.0 min
total: 4.7 %	current epoch: 99.2 %	loss: 0.000437	time estimate: 728.1 min
saved checkpoint at: ./ModelCheckpoints/c7o2h10/epoch_13

total: 4.7 %	current epoch: 3.2 %	loss: 0.000437	time estimate: 728.4 min
total: 4.7 %	current epoch: 6.4 %	loss: 0.000457	time estimate: 728.0 min
total: 4.7 %	current epoch: 9.6 %	loss: 0.000449	time estimate: 727.6 min
total: 4.7 %	current epoch: 12.8 %	loss: 0.000443	time estimate: 727.2 min
total: 4.7 %	current epoch: 16.0 %	loss: 0.000456	time estimate: 727.0 min
total: 4.7 %	current epoch: 19.2 %	loss: 0.000438	time estimate: 726.5 min
total: 4.7 %	current epoch: 22.4 %	loss: 0.000421	time estimate: 726.1 min
total: 4.8 %	current epoch: 25.6 %	loss: 0.000445	time estimate: 726.0 min
total: 4.8 %	current epoch: 28.8 %	loss: 0.000436	time estimate: 725.6 min
total: 4.8 %	current epoch: 32.0 %	loss: 0.00

In [ ]:
#deep_pot = DeepPotential().cuda()
#deep_pot.load_state_dict(torch.load('./ModelCheckpoints/epoch_13'))
Y_result = deep_pot.forward(X_test)

In [ ]:
result = backtransform(np.array(Y_result.data.tolist()), Y_min, Y_max)

In [ ]:
test_labels = backtransform(np.array(Y_test), Y_min, Y_max)

In [ ]:
np.mean(np.abs(test_labels - result))

In [ ]:
test_labels[:10]

In [ ]:
result[:10]

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
test_result = np.squeeze(backtransform(test_model(X_test), Y_min, Y_max).detach().cpu().numpy())

In [ ]:
val_result = np.squeeze(backtransform(Y_test, Y_min, Y_max).detach().cpu().numpy())

In [ ]:
np.mean(np.abs(test_result - val_result))

In [ ]:
val_result[:10]

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)